In [13]:
import pandas as pd
import numpy as np
import random

from typing import List, Tuple


In [3]:
df = pd.read_csv('../datasets/cleaned_RNN_dataset.zip')
df.head()

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,trees singing wind sky blue angels smiled saw ...,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,hard rock workout,3YouF0u7waJnolytf9JCXf,rock,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,na yeah spyderman freeze full effect uh huh re...,6oZ6brjB8x3GoeSYdwJdPc,Gold,back day r b new jack swing swingbeat rnb etc,3a9y4eeCJRmG9p4YKfqYIx,r&b,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,really can t stay baby cold outside i ve got g...,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,christmas soul,6FZYc2BvF7tColxO8PBShV,r&b,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,get business keep turning witness criminal fee...,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,k party dance mix,37i9dQZF1DX4RDXswvP6Mj,pop,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,hold breath look down keep trying darling okay...,3GNzXsFbzdwM0WKCZtgeNP,Chapters,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,...,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [ ]:
rap_df = df[df['playlist_name'] == 'southern hip hop']
cols = ['lyrics', 'track_album_id', 'track_album_id', 'language']
rap_df = rap_df.drop(cols, axis=1)
rap_df.head()

In [6]:
rap_df.shape

(211, 24)

In [8]:
rap_df.describe()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000
mean,9235.213270,0.743720,0.691355,5.521327,-6.868915,0.554502,0.248878,0.110417,0.004481,0.228555,0.595882,120.858682,268436.014218
std,5352.437027,0.128614,0.138430,3.798906,2.445386,0.498203,0.108257,0.125760,0.050035,0.177515,0.193633,33.498699,56350.903962
min,72.000000,0.286000,0.330000,0.000000,-15.538000,0.000000,0.033500,0.000047,0.000000,0.027700,0.115000,72.437000,133933.000000
25%,4639.000000,0.678000,0.589000,1.500000,-8.077000,0.000000,0.174000,0.016150,0.000000,0.096600,0.464500,88.961000,235160.000000
50%,9257.000000,0.783000,0.697000,6.000000,-6.419000,1.000000,0.250000,0.062900,0.000000,0.162000,0.622000,122.208000,264507.000000
75%,13713.000000,0.834500,0.806000,9.000000,-5.300500,1.000000,0.321000,0.167500,0.000002,0.325500,0.728500,150.027000,301020.000000
max,18104.000000,0.974000,0.963000,11.000000,-0.804000,1.000000,0.605000,0.626000,0.718000,0.822000,0.977000,191.692000,496133.000000


audio_cols is a list of the audio metrics I want to work with.

In [25]:
audio_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
55          0.663   0.606    5    -7.552     1        0.203        0.5790   
63          0.735   0.844    1    -5.215     1        0.167        0.1420   
89          0.397   0.740    3    -6.584     0        0.143        0.1050   
164         0.856   0.707    6    -7.041     0        0.178        0.0917   
166         0.855   0.698    2    -9.000     1        0.331        0.1750   

     instrumentalness  liveness  valence    tempo  duration_ms  
55                0.0     0.217    0.613   79.721       325373  
63                0.0     0.319    0.742  156.069       286427  
89                0.0     0.150    0.431  174.039       229347  
164               0.0     0.306    0.781  137.997       305440  
166               0.0     0.651    0.533   92.888       496133  


The function takes a list of audio metrics arrays for a playlist as input and returns a tuple of input/output sequences for training the RNN model. The input sequences are stored in a 3D numpy array X with shape (N-1, M, N-1), where N is the number of songs in the playlist and M is the number of audio metrics. Each input sequence consists of all but one of the audio metrics arrays in the playlist. The output sequences are stored in a 2D numpy array Y with shape (N-1, M), where each row corresponds to the audio metrics of the song that was left out of the corresponding input sequence.

In [60]:
def generate_input_output_pairs(audio_metrics: np.ndarray, target_song_index: int, target_song_audio_metrics: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Given a list of audio metrics for a playlist, generates input/output pairs
    for predicting the audio metrics of each song in the playlist based on the
    audio metrics of the other songs in the playlist.

    Args:
    - audio_metrics: A 2D numpy array of shape (N, M), where N is the number of
      songs in the playlist and M is the number of audio metrics.
    - target_song: The index of the song to predict.

    Returns:
    - A tuple (X, Y), where X is a 3D numpy array of input sequences with shape
      (N-1, M, N-1), and Y is a 2D numpy array of output sequences with shape
      (N-1, M).
    """
    # Get the number of songs and audio metrics
    N, M = audio_metrics.shape
    print(f"Audio Metrics shape, number of rows: {N}, number of columns: {M}")

    # Initialize the input and output arrays
    X = np.zeros((N-1, M, N-1))
    Y = np.zeros((N-1, M))

    # Generate the input/output pairs for each song
    for i in range(N):
        if i == target_song_index:
            continue

        # Get the indices of the other songs in the playlist
        other_songs = [j for j in range(N) if j != i and j != target_song_index]

        # Generate the input sequence
        
        X_i = audio_metrics[other_songs][:, :, np.newaxis]
        print(len(X_i))

        # Generate the output sequence
        # Y_i = audio_metrics[i]   <---- CHANGE THIS to target metrics most likely

        # Insert the input/output pair into the X and Y arrays

        # Holds all the songs and their corresponding audio metrics except for the target song
        # X[:, :, i-1] = X_i      <------ CHECK THIS
        # X[:, :, i-1] = X_i
        # print(f"Length of X data {len(X)}")

        # Containing the output sequence for the current song, 
        # Y[:, i-1] = Y_i        <------- CHECK THIS

    return X, Y



In this example, we're grouping the rows of the DataFrame df by the playlist_id column, and iterating over each group. Within the loop, we access the playlist name and genre by taking the first values of the playlist_name and playlist_genre columns of the current group (since these values will be the same for all rows in the group). We also extract the audio metrics for the songs in the playlist using the to_numpy method of a sub-DataFrame containing only the relevant columns. Finally, we generate the input/output pairs for the current playlist using the generate_input_output_pairs function, and can then proceed to train the RNN model on the resulting data.

In [61]:
import random

# Group the DataFrame by playlist_id
# This is 353 different groups
grouped = df.groupby('playlist_id')

# Loop through each playlist
for playlist_id, playlist in grouped:
    # Get the number of songs in the playlist
    num_songs = len(playlist)

    # Printing out the current playlist
    playlist_name = df.loc[df['playlist_id'] == playlist_id]['playlist_name'].iloc[0]
    print(f"Current Playlist being iterated through: {playlist_name}")
    
    # Generate a random integer between 0 and num_songs-1
    target_song_index = random.randint(0, num_songs-1)
    print('-'*50)
    print(f"Target Index value {target_song_index}")
    print('-'*50)
    
    # Get the audio metrics of the target song
    target_song_metrics = playlist.iloc[target_song_index][audio_cols].values
    print('$'*50)
    print(type(target_song_metrics))
    print(f"Target Song Metrics {target_song_metrics}")
    print('$'*50)
    # Generate the input/output pairs for each song in the playlist
    for i, row in playlist.iterrows():
        # Skip the target song
        if i == target_song_index:
            continue
        
        # Get the audio metrics of the current song
        audio_metrics = np.array(row[audio_cols]).reshape(1, -1)
        print('&'*50)
        print(f"Audio Metrics for first row in X column: {audio_metrics[0]}")
        print('&'*50)
        # Generate the input/output pairs for the current song
        # X represents the input sequence, Y represents the output sequence
        X, Y = generate_input_output_pairs(audio_metrics=playlist.loc[:, audio_cols].values, target_song_index=target_song_index, target_song_audio_metrics=target_song_metrics)
        
        # Train the RNN model on the input/output pairs
        # ...



Current Playlist being iterated through: hip hop n rnb
--------------------------------------------------
Target Index value 33
--------------------------------------------------
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<class 'numpy.ndarray'>
Target Song Metrics [0.68 0.843 1 -4.919 1 0.241 0.22 3.02e-05 0.378 0.743 81.958 192067]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Audio Metrics for first row in X column: [0.459 0.925 5 -1.327 0 0.35 0.248 0.0 0.174 0.5329999999999999 104.855
 357920]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Audio Metrics shape, number of rows: 68, number of columns: 12
66


ValueError: could not broadcast input array from shape (66,12,1) into shape (67,12)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15405 entries, 0 to 15404
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15405 non-null  int64  
 1   track_id           15405 non-null  object 
 2   track_name         15405 non-null  object 
 3   track_artist       15405 non-null  object 
 4   lyrics             15405 non-null  object 
 5   track_album_id     15405 non-null  object 
 6   track_album_name   15405 non-null  object 
 7   playlist_name      15405 non-null  object 
 8   playlist_id        15405 non-null  object 
 9   playlist_genre     15405 non-null  object 
 10  playlist_subgenre  15405 non-null  object 
 11  danceability       15405 non-null  float64
 12  energy             15405 non-null  float64
 13  key                15405 non-null  int64  
 14  loudness           15405 non-null  float64
 15  mode               15405 non-null  int64  
 16  speechiness        154